We will take the values generated by the GA and run them through gradient descent for chamfer distance and material separately. These points will hopefully generate new sets of non-dominated solution

In [50]:
#We start by installing libraries and importing some supporting code from a Github repository which will be used for linkage optimization
!pip install pymoo==0.6 svgpath2mpl
!git clone https://github.com/ahnobari/MOOCP.git

fatal: destination path 'MOOCP' already exists and is not an empty directory.


In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pymoo
import svgpath2mpl
# Utiliies for working with linkages (DeCoDE Lab)
from linkage_utils import *

# Other modules
import json
from IPython.display import HTML
import matplotlib.animation as animation
from scipy.optimize import minimize

In [52]:
# Initialize an empty list to store target curves
target_curves = []

# Loop to read 6 CSV files and store data in target_curves list
for i in range(6):
    # Load data from each CSV file and append it to the list
    target_curves.append(np.loadtxt('./data/%i.csv'%(i),delimiter=','))

# Get target curve 0 and plot
target_index = 0
target_pc = np.array(target_curves[target_index])

In [53]:
result_x = []
file_path = f'./results/{target_index}.csv'

# Open the CSV file
with open(file_path, 'r') as csvfile:
    # Create a CSV reader
    csv_reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for row in csv_reader:
        result_x.append(row)

In [54]:
result_x = [[float(value) for value in sublist] for sublist in result_x]

In [55]:
mechanisms = []
for x in result_x: 
    C, x0, fixed_nodes, motor, target  = from_1D_representation(x)
    valid, CD, mat, CD_grad,mat_grad,g_c = functions_and_gradients(C,x0,fixed_nodes,target_pc, motor)
#     res = minimize(CD,x0.reshape(-1),jac=CD_grad,method='BFGS')
    res = minimize(mat,x0.reshape(-1),jac=mat_grad,method='BFGS', tol=1E-8)
    mechanisms.append(to_final_representation(C,res.x.reshape(5,2),fixed_nodes,motor,target))


In [56]:
import csv
csv_file_path = './results_GD/{}_CD.csv'.format(target_index)
# csv_file_path = './results_adam/{}_mat.csv'.format(target_index)

data_to_write = []
for x in mechanisms:
    data_to_write.append(list(x))

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    # Create a CSV writer
    csv_writer = csv.writer(csvfile)

    # Write the data
    csv_writer.writerows(data_to_write)
